In [1]:
import leafmap
from samgeo import SamGeo, tms_to_geotiff
import numpy as np
from typing import Dict, Any, List, Optional


In [2]:
center = [-96.78779636758789,32.78009026607563]
map_zoom = 20
download_zoom = 22
box_threshold = 0.24
image = "Image.tif"
points_include = [[-96.78735903257792,32.76919652798132],[-96.78694633366432,32.76941409030799],[-96.78693217686015,32.76888747455614],[-96.78506568738678,32.76824808468834]]

points_exclude = []
point_coords = points_include + points_exclude
# The model type. It can be one of the following: vit_h, vit_l, vit_b. Defaults to 'vit_h'. See https://bit.ly/3VrpxUh for more details.
# vit_b, base - 86M,
# vit_l, large - 307M,
# vit_h, huge - 632M
model_type = "vit_h" # can be vit_h, vit_b, vit_l, //vit_tiny


In [3]:

m = leafmap.Map(center=center, zoom=map_zoom, height="800px")
m.add_basemap("SATELLITE")

In [4]:
def calculate_bounding_box(points: List[List[float]], buffer_size: float) -> List[float]:
    """
    Calculate bounding box from points with buffer.
    
    Args:
        points: List of [longitude, latitude] coordinates
        buffer_size: Buffer size in degrees
        
    Returns:
        [min_lon, min_lat, max_lon, max_lat]
    """
    points_array = np.array(points)
    min_lon = points_array[:, 0].min() - buffer_size
    min_lat = points_array[:, 1].min() - buffer_size
    max_lon = points_array[:, 0].max() + buffer_size
    max_lat = points_array[:, 1].max() + buffer_size
    
    return [min_lon, min_lat, max_lon, max_lat]


In [5]:

bbox = calculate_bounding_box(point_coords, 0.001)
print(image, bbox, download_zoom, "Satellite", True)

Image.tif [np.float64(-96.78835903257793), np.float64(32.76724808468834), np.float64(-96.78406568738677), np.float64(32.77041409030799)] 22 Satellite True


In [6]:
tms_to_geotiff(output=image, bbox=bbox, zoom=download_zoom, source="Satellite", overwrite=True)

Downloaded image 0001/2295
Downloaded image 0002/2295
Downloaded image 0003/2295
Downloaded image 0004/2295
Downloaded image 0005/2295
Downloaded image 0006/2295
Downloaded image 0007/2295
Downloaded image 0008/2295
Downloaded image 0009/2295
Downloaded image 0010/2295
Downloaded image 0011/2295
Downloaded image 0012/2295
Downloaded image 0013/2295
Downloaded image 0014/2295
Downloaded image 0015/2295
Downloaded image 0016/2295
Downloaded image 0017/2295
Downloaded image 0018/2295
Downloaded image 0019/2295
Downloaded image 0020/2295
Downloaded image 0021/2295
Downloaded image 0022/2295
Downloaded image 0023/2295
Downloaded image 0024/2295
Downloaded image 0025/2295
Downloaded image 0026/2295
Downloaded image 0027/2295
Downloaded image 0028/2295
Downloaded image 0029/2295
Downloaded image 0030/2295
Downloaded image 0031/2295
Downloaded image 0032/2295
Downloaded image 0033/2295
Downloaded image 0034/2295
Downloaded image 0035/2295
Downloaded image 0036/2295
Downloaded image 0037/2295
D

In [7]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")

In [8]:
sam = SamGeo(
    model_type=model_type,  
    automatic=False,
    sam_kwargs=None,
)

# change the model_type from vit_h to vit_b, so it take less space.
sam.set_image(image)

Segment the image with the point coordinates.

In [10]:
sam.predict(points_include, point_labels=1, point_crs="EPSG:4326", output="mask1.tif")


In [11]:
m.add_raster("mask1.tif", cmap="viridis", nodata=0, layer_name="Mask")
m

Map(center=[32.768831000000006, -96.7862125], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…